In [ ]:
report = "/home/rguan/project/ML-Capstone-Team-7/FinTSBridge_models/TSLib_baseline/results_WTI_trading_extend_20251129_061059/WTI_trading_report_rankings.csv"

In [2]:
# prepare_period_datasets.py
import pandas as pd

# 读取原始数据
df = pd.read_csv('./dataset/FBD/WTI-log.csv')
df['date'] = pd.to_datetime(df['date'])

# 创建不同时期的数据集
periods = {
    'WTI-log-2010.csv': '2010-01-01',
    'WTI-log-2015.csv': '2015-01-01', 
    'WTI-log-2020.csv': '2020-01-01',
}

for filename, start_date in periods.items():
    subset = df[df['date'] >= start_date].copy()
    subset.to_csv(f'./dataset/FBD/{filename}', index=False)
    print(f"{filename}: {len(subset)} rows ({subset['date'].min()} to {subset['date'].max()})")


## 预期结果与解读

WTI-log-2010.csv: 3975 rows (2010-01-04 00:00:00 to 2025-10-21 00:00:00)
WTI-log-2015.csv: 2717 rows (2015-01-02 00:00:00 to 2025-10-21 00:00:00)
WTI-log-2020.csv: 1458 rows (2020-01-02 00:00:00 to 2025-10-21 00:00:00)
